In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os

# Step 1: Explore the Time_domain_subsamples folder
print(" Checking dataset structure...")
subsample_path = '/content/3.Time_domain_subsamples'

# List what's inside
!ls -la /content/3.Time_domain_subsamples/ | head -20

# Find all CSV files in the subsamples folder
csv_files = glob('/content/3.Time_domain_subsamples/**/*.csv', recursive=True)

print(f"\n✅ Found {len(csv_files)} CSV files")
print("\nFirst 10 files:")
for i, file in enumerate(csv_files[:10]):
    print(f"  {i+1}. {file}")

 Checking dataset structure...
ls: cannot access '/content/3.Time_domain_subsamples/': No such file or directory

✅ Found 0 CSV files

First 10 files:


In [ ]:
import pandas as pd
from glob import glob
from pathlib import Path

# Get all CSV files from Time_domain_subsamples
csv_files = glob('/content/3.Time_domain_subsamples/**/*.csv', recursive=True)

print(f"Loading {len(csv_files)} files...")

df_list = []

for i, csv_file in enumerate(csv_files):
    try:
        # Read the CSV
        df_temp = pd.read_csv(csv_file)

        # Extract metadata from file path
        # Example path: /content/3.Time_domain_subsamples/Participant_1/walk/sample_001.csv
        path_parts = Path(csv_file).parts

        # Find participant (e.g., "Participant_1")
        participant = None
        activity = None

        for part in path_parts:
            if 'Participant' in part or 'participant' in part:
                participant = part
            elif part.endswith('.csv'):
                # Parent folder is usually the activity
                idx = path_parts.index(part)
                if idx > 0:
                    activity = path_parts[idx - 1]

        # Add metadata columns
        df_temp['participant'] = participant if participant else 'Unknown'
        df_temp['activity'] = activity if activity else Path(csv_file).stem
        df_temp['file_path'] = csv_file

        df_list.append(df_temp)

        # Progress update every 100 files
        if (i + 1) % 100 == 0:
            print(f"  Loaded {i + 1}/{len(csv_files)} files...")

    except Exception as e:
        print(f"⚠️ Error loading {csv_file}: {e}")

# Combine all dataframes
df = pd.concat(df_list, ignore_index=True)

print("\n" + "="*60)
print("✅ DATASET LOADED SUCCESSFULLY!")
print("="*60)
print(f"Total samples: {len(df):,}")
print(f"Total features: {len(df.columns)}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nShape: {df.shape}")

Loading 0 files...


ValueError: No objects to concatenate